In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
#from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
#from xgboost import XGBClassifier
from sklearn.metrics import classification_report, roc_auc_score

In [2]:
data = pd.read_csv('../data/Telco-Customer-Churn.csv')
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [3]:
df = data
data.isna().sum()
df.drop("customerID", axis=1, inplace=True)
df['Churn'] = df['Churn'].map({'Yes': 1, "No":0})

In [4]:
X = df.drop('Churn', axis=1)
y = df['Churn']

In [5]:
numeric_features = ['tenure', 'MonthlyCharges', 'TotalCharges']
binary_features = [col for col in X.columns if X[col].nunique() == 2 and col not in numeric_features]
categorical_features = list(set(X.select_dtypes(include=['object']).columns) - set(binary_features))

In [6]:
from sklearn.preprocessing import LabelEncoder, FunctionTransformer

In [7]:
numeric_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

def binary_encode(X):
    return X.applymap(lambda x: 1 if x == 'Yes' else 0)

binary_pipeline = Pipeline([
    ("binary_encoder", FunctionTransformer(binary_encode))
])

categorical_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown='ignore'))
])

In [8]:
preprocessor = ColumnTransformer([
    ("num", numeric_pipeline, numeric_features),
    ("bin", binary_pipeline, binary_features),
    ("cat", categorical_pipeline, categorical_features)
])

model_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", RandomForestClassifier(random_state=42))
])

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

model_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['tenure', 'MonthlyCharges',
                                                   'TotalCharges']),
                                                 ('bin',
                                                  Pipeline(steps=[('binary_encoder',
                                                                   FunctionTransformer(func=<function binary_encode at 0x000001FFC4943060>))]),
                                                  ['gender', 'SeniorCitizen',
                                                   'Par...
                                                   'PaperlessBilling']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['OnlineBackup',
                                                   'MultipleLines',
                                                   'TechSupport', 'StreamingTV',
                                                   'DeviceProtection',
                                                   'OnlineSecurity',
                                                   'PaymentMethod',
                                                   'InternetService',
                                                   'Contract',
                                                   'StreamingMovies'])])),
                ('classifier', RandomForestClassifier(random_state=42))])

In [10]:
y_pred = model_pipeline.predict(X_test)
y_proba = model_pipeline.predict_proba(X_test)[:,1]

print("Classification Report:")
print(classification_report(y_test, y_pred))
print("ROC-AUC Score:", roc_auc_score(y_test, y_proba))

Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.88      0.86      1035
           1       0.61      0.49      0.54       374

    accuracy                           0.78      1409
   macro avg       0.72      0.69      0.70      1409
weighted avg       0.77      0.78      0.77      1409

ROC-AUC Score: 0.8126482213438737


In [11]:
param_grid = {
"classifier__n_estimators": [100, 200],
"classifier__max_depth": [5, 10]
}

grid = GridSearchCV(model_pipeline, param_grid, cv=3, scoring="roc_auc")
grid.fit(X_train, y_train)

print("Best Parameters:", grid.best_params_)
print("Best ROC-AUC:", grid.best_score_)

Best Parameters: {'classifier__max_depth': 5, 'classifier__n_estimators': 200}
Best ROC-AUC: 0.8453890154777352


In [12]:
import joblib
joblib.dump(grid.best_estimator_, "../models/churn_model.pkl")

['../models/churn_model.pkl']

In [13]:
pipeline_loaded = joblib.load("../models/churn_model.pkl")
sample_prediction = pipeline_loaded.predict(X_test.iloc[[0]])
print("Prediction for first test customer:", sample_prediction)

Prediction for first test customer: [0]
